In [1]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from keras.models import Model
import time
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import math
from tensorflow import keras 
import os
from PIL import Image, ImageOps, ImageChops
import numpy as np

In [2]:
def loadTestData(path):
    filelist=os.listdir(path)
    
    all_images = []
    all_names = []
    
    # Iterate through class folders in path
    for current_image in filelist:
        
        all_names.append(current_image)
        
        temp = Image.open(path+"/"+current_image)
        keep = temp.copy()
        all_images.append(keep)
            
    return all_images, all_names

In [3]:
#train_images, train_labels, train_class_list 
all_images, all_names = loadTestData("Data/test_data")

In [4]:
len(all_images)

8730

In [5]:
len(all_names)

8730

## Preprocessing der Testdaten

In [6]:
SIZE = 100

In [7]:
def resizeImages(list_images):
    # Daten einheitlich auf 120x120 hochskalieren
    for i in range(len(list_images)):
        if list_images[i].width != SIZE:
            #display(list_images[i])
            new_image = list_images[i].resize((SIZE, SIZE))
            #display(new_image)

            list_images[i] = new_image
    return list_images

In [8]:
X_test = resizeImages(all_images)

In [9]:
def convertImageToNp(list_images):
    all_np_images = []
    for image in list_images:
        current_image = np.array(image)
        

        # 4. wert bei rgb weg machen
        current_image = current_image[:, :, :3]
        #print(current_image.shape)

        all_np_images.append(current_image)

    return np.array(all_np_images)

In [10]:
X_test = convertImageToNp(X_test)


In [11]:
X_test.shape

(8730, 100, 100, 3)

## Model laden

In [41]:
model_name = "xceptionizerV4-nachtr2_8136"
rec_modelBEST = keras.models.load_model("goodModels/"+model_name)

In [42]:
model_name = "vgg16_nachtr_7823_BEST-v1-77.41"
rec_model2Best = keras.models.load_model("goodModels/"+model_name)

In [43]:
model_name = "vgg16_MisterDropout-v3-75.91"
rec_model3Best = keras.models.load_model("goodModels/"+model_name)

In [44]:
model_name = "resnet50_nachtr2_80.53"
rec_model4Best = keras.models.load_model("goodModels/"+model_name)

In [45]:
y_pred1 = rec_modelBEST.predict(X_test)

273/273 [==============================] - 3s 9ms/step


In [46]:
y_pred2 = rec_model2Best.predict(X_test)

273/273 [==============================] - 4s 12ms/step


In [47]:
y_pred3 = rec_model3Best.predict(X_test)

273/273 [==============================] - 3s 11ms/step


In [48]:
y_pred4 = rec_model4Best.predict(X_test)

273/273 [==============================] - 4s 12ms/step


In [55]:
final_pred = (y_pred1+y_pred2+y_pred3+y_pred4)/4

In [56]:
y_pred = final_pred

In [57]:
import pandas as pd

In [58]:
df_pred = pd.DataFrame(y_pred,
                   columns=["Bicycle","Bridge","Bus","Car","Chimney","Crosswalk","Hydrant","Motorcycle","Other","Palm","Stair","Traffic Light"])

df_names = pd.DataFrame(all_names,
                   columns=["ImageName"])

df_full = pd.merge(df_names, df_pred, left_index=True, right_index=True)

In [59]:
df_full

,ImageName,Bicycle,Bridge,Bus,Car,Chimney,Crosswalk,Hydrant,Motorcycle,Other,Palm,Stair,Traffic Light
0,03000.png,5.247577e-04,1.537159e-05,0.631155,0.364734,4.268799e-06,0.000681,5.234825e-06,2.095778e-06,0.000093,2.746655e-03,6.295043e-06,0.000033
1,03001.png,5.149589e-02,3.784669e-03,0.001143,0.420824,7.543443e-04,0.384109,9.255276e-04,3.174576e-03,0.114180,2.299610e-03,1.520118e-03,0.015788
2,03002.png,1.743689e-08,1.774650e-07,0.000304,0.972631,9.440081e-10,0.027062,8.777469e-09,1.390918e-07,0.000002,2.860974e-07,9.831463e-10,0.000001
3,03003.png,6.539125e-04,1.286151e-04,0.777649,0.000034,2.354194e-05,0.205208,5.254985e-07,2.105750e-07,0.000002,1.417052e-02,2.093131e-03,0.000037
4,03004.png,1.210047e-03,3.996750e-03,0.050061,0.172336,3.939630e-04,0.028864,3.894994e-04,6.516040e-04,0.738256,2.182879e-03,1.063507e-03,0.000594
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8725,11726.png,1.731811e-02,1.649997e-05,0.515693,0.088856,1.967161e-05,0.015859,1.622729e-05,4.663106e-03,0.000340,6.688482e-06,3.570476e-01,0.000164
8726,11727.png,1.915421e-05,1.486833e-03,0.000139,0.001291,1.944986e-04,0.000026,6.607836e-06,1.714033e-06,0.956784,3.931430e-02,6.653789e-05,0.000670
8727,11728.png,4.303919e-06,1.242667e-04,0.983710,0.009521,2.688308e-05,0.000821,3.192810e-06,9.855545e-07,0.002836,2.537235e-03,2.750054e-04,0.000140
8728,11729.png,3.277080e-05,6.229158e-05,0.003257,0.995805,1.927431e-07,0.000675,3.233587e-05,1.038864e-04,0.000018,4.920988e-06,2.172987e-06,0.000007


In [60]:
df_full.to_csv("prediction/MASTER"+model_name+'_prediction.csv',sep=',', float_format='%.8f', index=False) 